<span type="title">多态</span> | <span type="update">2018-07-20</span> - Version <span type="version">1.3</span>
    
    
<span type="intro"><p class="card-text">本章主要讲解多态。多态是一种由于向上转型行为和动态绑定机制带来的类重用中继承的一组规范行为表现。这种行为表现可以让我们在编程时拥有丰富且规范的数据结构，操纵一套基类的API，就可以控制任意由基类继承的类型，获得不同的行为。</p><p class="card-text">在第一部分，讲解了向上转型、动态绑定有关的知识。在向上转型部分，探讨了可能发生转型的时机。在动态绑定部分，讲解了动态方法调用的机制（基类实现，子类扩展）、其带来的巨大的扩展性，同时分析了静态方法、域的不可绑定的特性，最后探讨了绑定后调用方法的陷阱问题。</p><p class="card-text">在第二部分，讲解了构造器和多态的关系，重复提到了构造器在子类构造的顺序、构造器内部可能发生的向上转型和逻辑错误。最后介绍了协变类型、利用继承进行设计的理念。继承常常被用来表示不同的状态差异，而是用组合将继承实例化后，调用基类接口，使用多态可以表示自己的状态和变化。由于多态（转型和绑定），我们现在只需要基类方法，就可以扩展出丰富而又有序的程序模块。在最后的最后，探讨了基类和导出类的关系，是否要扩展导出类以及向下转型的一些细节。</p></span>

面向对象的程序设计语言中，三大基本特征是 `数据抽象、继承和多态`。

其中对于抽象而言，就是将对象的特征和行为封装起来。“封装” 通过这种归类来创造新的数据类型 `xxx.class`，其构造了新的数据类型。“实现隐藏” 通过权限关键字实现了对于接口和实现（细节）的分离。而继承，则提供了一种代码重用的方法，而在代码重用的时候，最容易出现的问题就是类型之间的耦合（父类、子类和它类），因此，“多态” 就是解决这种耦合的的重要工具。

这是一种重要的能力，它允许将由同一个类继承而来的多个类型当作同一类型来处理，因此，只用写一份代码，就可以在由这个类型导出的任何类型上使用。从某种程度上，继承和多态的关系，是“变与不变”的关系。

因此，我觉得对于Java的OO，三大基本特征应该是 `数据抽象、实现隐藏、继承多态`

简而言之，多态是一种由于向上转型行为和动态绑定机制带来的类重用中继承的一组规范行为表现。这种行为表现可以让我们在编程时拥有丰富且规范的数据结构，操纵一套基类的API，就可以控制任意由基类继承的类型，获得不同的行为。

# 向上转型

向上转型来自于UML的箭头，本来是从父类指向子类的箭头反向上指，因此称作向上转型。向上转型允许我们定义一个基类的类型，然后使用子类的构造器创建一个对象，这个对象将“向上转型”为基类使用。

**转型发生的位置**

转型一般发生在以下位置：
- 定义处 `Father x = new Son();`
- 方法参数处 `methodA(Father x); metod(new Son());`
- 类型转换处 `Son a = Son();(Father)a;`
- 构造器调用父类构造器的时候... `super();`

其实在任何时候，只要类型不匹配还能够正确调用的，肯定是发生了转型。转型还有可能发生在构造器调用父类构造器的时候。只要任何箭头从子类指向父类的情况，都发生了转型。转型是一种权力关系的更改，向上转型是权力缩小。向下转型是权力扩大，不过因为子类可能实现了其他类，因此可能出现错误。Java对所有向下转型都检查类型。

```java
//show how to use up casting
import static com.mazhangjing.Print.*;
public class Game {
	private Cycle cycle;
	public Game(Cycle c){
		this.cycle = c;
	}
	public void ride(){
		print("In Game.rideBy()");
		cycle.ride();
	}
	public static void main(String[] args){
		Game a = new Game(new Unicycle());
		a.ride();//up casting
	}
}
class Cycle {
	public void ride() {
		print("Cycle ride");
		wheels();
	}
	public void wheels() {
		print("The cycle have 4 wheels");
	}
}
class Unicycle extends Cycle {
	public void ride() {
		print("Unicycle ride");
	}
}
class Bicycle extends Cycle {
	public void ride() {
		print("Bicycle ride");
	}
}
class Tricycle extends Cycle {
	public void ride() {
		print("Tricycle ride");
	}
}

In Game.rideBy()
Unicycle ride
```


如上所示，在 Game 的 ride 方法中，其接受的是 Cycle 类，而现在处理的是 Unicycle 类，这就发生了向上转型。向上转型里发生的事情很有意思，当基类和子类共有一个方法被调用，ride显示了其结果——子类的方法被返回，因此这种机制可以让我们只关注基类，而不用担心导出类，当导出类重载其方法时，返回导出类方法，反之，则返回基类方法。

# 动态绑定

## 动态绑定概要

当进行向上转型的时候，其实发生了动态绑定。

在C语言中，所有的代码在被编译的时候就已经绑定了，其称之为前向绑定，因此方法体.方法名是唯一的，指向唯一一段代码，在程序运行期间不可更改。但是Java的绑定（方法体和方法名）是动态，或者说是运行时绑定，编译器在编译的时候一直不知道对象的类型，但是在之后，方法调用机制会判断正确对象的类型，并且调用恰当的方法。

Java中除了 static 和 final(private属于final)之外，都是动态绑定的。在早期，可能将一个方法声称为 final 为了获得更好的性能，关闭动态绑定，但是，这对于程序整体性能影响不大。这一点需要注意。

对于上述例子，**向上转型**指的是：

`Cycle x = new Unicycle()` 就是一个动态绑定的例子，在这个声明中，当 `Unicycle()` 返回一个对象的时候，因为通过继承 Unicycle 本身就是一种 Cycle，因此这句话没有错误。这是一个向上转型。

**动态绑定**指的是：

`x.ride()` 现在，调用了 `ride()` 方法，因为在上一句我们知道 x 一定是 Cycle的一个引用（根据语法定义），那么这个 `ride` 必须在 `Cycle` 基类被实现，但是真正调用的时候，也就是这个 `.`符号 起作用的时候，Java会寻找合适的一个对象，结果它找到了`Unicycle` 这个对象的一个实例 `x`，因此，就使用这个实例来进行 `ride` 的方法调用。

在这里就发生了向上转型和动态绑定。

需要注意，对于这个方法，一定在基类中被实现，对于这个方法的真实调用，如果子类进行了重载，那么会因为动态绑定使用子类的对象实例，因此使用了子类的重载过的方法。这一点尤其需要注意，**动态绑定方法必须在基类中实现，动态绑定方法可在子类重载**。看下面这个例子：

```java
//use to show dynamic binding
import static com.mazhangjing.Print.*;
import java.util.*;
public class Shapes {
	public static void main(String[] args){
		RandomShape rs = new RandomShape();
		Shape[] s = new Shape[5];
		for (int i = 0 ; i < s.length; i++){
			s[i] = rs.next();
		}
		for (Shape i : s){
			i.draw();
			i.author();
		}
		Shape c = new Cycle(4);
		//print(c.wheels());//error, c is a Shape quote, the compiler will
		//first find wheels() in Shape class, and because of dynamic binding
		//it will call Cycle().xxx method, the compiler don't know it.
		c.draw(); // right
	}
}
class Shape {
	public void draw() {print("Shape is drawing...");}
	public void author() {print("By Corkine Ma");}
}
class Line extends Shape {
	@Override public void draw() {print("Line is drawing...");}
	public void author() {print("By Corkine Ma in Line");}
}
class Cycle extends Shape {
	private int wheels_num;
	Cycle(int num){
		wheels_num = num;
	}
	@Override public void draw() {print("Cycle is drawing...");}
	public int wheels(){
		return wheels_num;
	}
}
class Triangle extends Shape {
	public void draw() {print("Triangle is drawing...");}
}
class RandomShape {
	private Random rand = new Random();
	public Shape next(){
		switch (rand.nextInt(3)){
			default:
			case 0: return new Line();
			case 1: return new Cycle(rand.nextInt(3));
			case 2: return new Triangle();
		}
	}
}
```

**向上转型**: `RandomShape`的`next()`方法在`Shapes`调用时返回的是`Shape`类型，在真实调用时，不论是`Line`还是`Cycle`还是`Triangle`，都发生了向上转型。

**动态绑定**: 在`draw`方法发生的时候，则进行了动态绑定，调用子类实例方法体，返回子类中重载的方法。

需要注意注释的地方，如果一个方法在子类中有，但在父类中没有，那么不可调用，这很简单，因为这里的c是一个Shape，编译器无法在Shape中找到这个wheels()的方法。这一点尤其需要注意。

**Java重载机制的变与不变**

因此，现在我们知道了变与不变，其中的变指的是动态绑定，给不同的方法体，可以**调用其内部不同的重载**。

不变指的是向上转型，向上转型暗含着这样的表述：**你的调用必须在基类中实现**，之后才能调用，而不同于wheels()方法。因为只有这样，我们在使用调用方法时，不用担心继承类是如何处理的，因为不论怎样，如果找不到重载，总是会返回基类的这个方法。

## 绑定的可扩展性

下面这个例子和上面类似，不过其包含了多层的继承，每个继承重载基类的方法也不同。对于这种接近真实的情况，可以看出Java重载机制——变与不变所带来的好处。从某种程度来说，这解决了类与类之间的过渡耦合问题。这总是在现实中经常发生——一个类依赖于另一个类的某个子类的重载方法——利用向上转型和动态绑定，我们就可以直接调用基类API，而不用关心子类是否实现、怎样实现重载的动态方法。

下面例子中`Instrument`表示乐器基类，其`play`方法接受一个`Note`曲谱。这个基类有一个`adjust()`方法来调弦。

现在我们通过继承构造了几种不同的乐器，比如`Wind，Percussion`等，对于`Wind`也进行了继承，这杯用来表示复杂的“变”的结构。在这些子孙类中，有一些重载了`adjust()`和`play()`方法，有一些没有，但不论怎样，在最后的`Music`类中，对`Instrument`基类调用`play()`方法总不会出错，因为如果在子类没有实现，那么回转回头调用基类的定义，因为我们已经要求过这种动态绑定必须在基类中实现，因此程序具有了丰富的可扩展性，并且不会出错。

注意 `print(orchestra[4]);` 这句话，在这个类中是没有`toString`实现的，但没有发生任何错误。

```java
//use to show dynamic binding advanced tech
import static com.mazhangjing.Print.*;
import java.util.*;
enum Note {
	MIDDLE_C, C_SHAPE, B_FLAT;
}
class Instrument {
	void play(Note n) {print("Playing Instrument "+n);}
	public String toString() {return "Instrument";}
	void adjust() {print("Adjusting Instrument");}
}
class Wind extends Instrument {
	void play(Note n) {print("Wind.play() " + n);}
	String what() {return "Wind";}
	void adjust() {print("Adjusting Wind");}
}
class Percussion extends Instrument {
	void play(Note n) {print("Percussion.play() " + n);}
	String what() {return "Percussion";}
	void adjust() {print("Adjusting Percussion");}
}
class Stringed extends Instrument {
	void play(Note n) {print("Stringed.play() " + n);}
	String what() {return "Stringed";}
	void adjust() {print("Adjusting Stringed");}
}
class Brass extends Wind {
	//void play(Note n) {print("Brass.play() "+n);}
	void adjust() {print("Adjusting Brass");}
}
class Woodwind extends Wind {
	void play(Note n) {print("Woodwind.play() "+n);}
	void adjust() {print("Adjusting Woodwind");}
}
public class Music {
	public static void tune(Instrument i){
		i.play(Note.MIDDLE_C);
	}
	public static void turnAll(Instrument[] e){
		for (Instrument i : e){
			tune(i);
		}
	}
	public static void main(String[] args){
		Instrument[] orchestra = {
			new Wind(),
			new Percussion(),
			new Stringed(),
			new Brass(),
			new Woodwind()
		};
		turnAll(orchestra);
		print(orchestra[4]);
		
		RandomIns x = new RandomIns();
		x.next().adjust();
	}
}
class RandomIns {
	private Random rand = new Random();
	public Instrument next() {
		switch (rand.nextInt(5)){
			default:
			case 0: return new Wind();
			case 1: return new Percussion();
			case 2: return new Stringed();
			case 3: return new Brass();
			case 4: return new Woodwind();
		}
	}
}

Wind.play() MIDDLE_C
Percussion.play() MIDDLE_C
Stringed.play() MIDDLE_C
Wind.play() MIDDLE_C
Woodwind.play() MIDDLE_C
Instrument
Adjusting Wind
```

## 基类私有方法不可绑定

下列代码展示了一个很有意思的问题， `Father x = new Fault()` 构造了一个 `Father` 类型的引用 `x`，其调用 `f()` 方法，按道理，会将子类的方法头放置在这里，调用 `Fault` 的 `f()` 方法。但是这里的问题是，由于父类的 `f()` 方法为私有，也就是说子类的 `f()` 方法并非来自父类的重载。因此，Java并不会找到一个重载的 `f()` 方法。

如果这里直接调用的是 `Fault` 类的 `f()` 的话，结果当然是没有重载的那个子类方法被调用。但是这里发生的事情其实是动态绑定，所以唯一可能的解释就是：Java在发现你使用动态绑定的时候，发现了一个 private，其实是一个 final 关键字，所以就截获了这个绑定调用，所以我们调用的不是子类的实例，也就是说，这个动态绑定没有发生，只有向上引用，因此，作为 `Father` 的引用，就正常的调用了父类的 `f()`。

```java
//private method fault in up casting
import static com.mazhangjing.Print.*;
class Fault extends Father {
    //@Override 
	public void f() {
		print("Public f() in Falut");
	}
	public void g() {
		print("Public g() in Falut");
	}
}
public class Father {
	private void f(){
		print("Private f() in father");
	}
	public void g(){
		print("Public g() in father");
	}
	public static void main(String[] args){
		Father x = new Fault();
		x.f(); //Private f() in father
		x.g(); //Public g() in Falut
	}
}
```

## 域和静态方法不可绑定

动态绑定不能去重载子类中的域和静态方法，从某种程度上来说，这算是一种缺陷。

```java
//show the static method and the field 's relationship with dynamic binding
import static com.mazhangjing.Print.*;
public class People extends Super {
	int i = 30;
	static void getInfo(){
		print("People.static getInfo method");
	}
	public void getInfo2(){
		print("People.public getInfo method");
	}
	public static void main(String[] args){
		Super x = new People();
		print(x.i);
		x.getInfo();
		x.getInfo2();
	}
}
class Super {
	int i = 20;
	static void getInfo() {
		print("Super.static getInfo method");
	}
	public void getInfo2(){
		print("Super.public getInfo method");
	}
}
20
Super.static getInfo method
People.public getInfo method
```

如上的代码所示，返回的是父类的域和父类的静态方法，而对于动态方法，则正确进行了绑定。

对于私有方法的覆盖、静态方法和域的无效的多态机制是Java固有的缺陷。因为按照绝对正确的模型，这样的写法会调用子类相关的域和静态方法，但是出于某种原因，当给方法名安上方法体时，Java只对那些动态方法进行了多态和动态绑定，而对于其余的——私有、静态方法和域，都默认按照基类进行处理。这和 private 方法类似。

因此，可以说，对于任何有 final 意味的玩意儿，都不能发生动态绑定，只到向上提升为止，因此所截获的都是基类的属性。

## 在绑定发生后调用其它方法

下面这个例子很好的说明了Java多态是如何进行的/机制问题（对于动态方法而言）。注意这里发生了动态绑定，因此，头被传递给基类对象，所以，在之后的这个方法中调用任意内容，都在子类体中进行的调用。当然，除非子类中没有，那么它会去隐性的子类继承的父类方法和域中去寻找。这看起来像是在父类中发生的代码，其实并不是，因为当你调用 `this.methodA()` 的时候清楚的看到了，这是在子类进行的。

```java
//use to show up casting advanced tech II
import static com.mazhangjing.Print.*;
class Father {
	void methodA(){
		print("Father-methodA");
	}
	void methodB(){
		this.methodA();
		print("Father-methodB");
	}
}
public class Son extends Father{
	@Override void methodA() {
		print("Son-methodA");
	}
	void findFather(Father f){
		f.methodB();
	}
	public static void main(String[] args){
		Son s = new Son();
		s.findFather(s);
	}
}
Son-methodA
Father-methodB
```

这个例子是这样进行的：调用 `findFather(s)` 发生向上引用 --> 调用 `f.methodB()` 发生动态绑定 --> 调用子类的 `methodB()`（注意，这里并非调用父类方法！！！）--> 在调用过程中进一步调用了子类的 `methodA()` --> 执行完毕。

# 构造器和多态

## 构造器调用顺序

构造器的目的在于，在内存中构建对象所需的内存空间，将一些东西初始化后放在内存某些位置。这并不是一个简单的工作。在之前介绍了多次构造器、静态变量的调用顺序问题，这里还要再提一次，因为现在有了继承。

子类的构造器在构造其自身时，如果不调用父类构造器，那么子类中依赖父类的代码将无法被正确初始化。因此，不论是显式调用父类构造器`super()`还是Java自动调用，当一个子类被调用时，首先其继承的所有类的构造器将会被调用，以确定这个子类所有变量、域和其余东西，当确定后才能正确的对其进行初始化。

下面的代码展示了其顺序问题：

```java
/xxx/Sandwich.java
//constructor order
import static com.mazhangjing.Print.*;
public class Sandwich extends PortableLunch{
	private Cheese c = new Cheese();
	private Pickle p = new Pickle();
	Sandwich() {print("Sandwich()");}
	public static void main(String[] args){
		Sandwich x = new Sandwich();
	}
}
class Meal {Meal() {print("Meal()");}}
class Bread {Bread() {print("Bread()");}}
class Cheese {Cheese() {print("Cheese()");}}
class Lettuce {Lettuce() {print("Lettuce()");}}
class Pickle extends Lettuce {Pickle() {print("Pickle()");}}
class Lunch extends Meal {Lunch(){print("Lunch()");}}
class PortableLunch extends Lunch 
	{PortableLunch(){print("PortableLunch()");}}
Meal()
Lunch()
PortableLunch()
Cheese()
Lettuce()
Pickle()
Sandwich()
```

其中可以看到，具有最高优先级的就是父类构造器，它会一直迭代寻找最初的基类，然后依次调用这些基类的构造器（因为一个类的构造器可以确保这个类的代码正确初始化），这样子类所依赖的所有代码都会正确被显示出来。这里的 `Meal() Lunch() PortableLunch()` 均是如此。

当类迭代完毕之后，静态变量就开始被创建，并存放入内存，我们看到了`Cheese()`。然后应该是 `Pickle`,因为这是一个子类，因此其也要先调用父类方法，所以有了`Lettuce()，Pickle()`，最后，才是自己的构造器`Sandwich()`。

## 构造器和继承类的清理

之间讲过，对于继承而言，其清理必须和构造相反，清理不仅需要清理自己类创建的对象，还需要调用父类的清理方法，来清理父类所创建的对象。`super.dispose()`

## 构造器内部的多态行为

之前说过，向上提升发生在定义处、直接类型转换处、参数传递处。起始，在构造器内部也可能发生向上提升，这种错误难以发觉。如下：

```java
/PolyConstructors.java
//show why you should make constructor simple
import static com.mazhangjing.Print.*;
class Glyph {
	//private
    void draw() {print("Glyph.draw()");}
	Glyph() {
		print("Glyph() before draw()");
		draw();
		print("Glyph() after draw()");
	}
}
class RectangularGlyph extends Glyph {
	private int degree = 20;
	RectangularGlyph(int d){
		degree = d;
		print("RectanglarGlyph Constructor. degree is "+degree);
	}
	void draw() {
		print("RectanglarGlyph draw. degree is "+degree);
	}
}
public class PolyConstructors {
	public static void main(String[] args){
		new RectangularGlyph(30);
	}
}
Glyph() before draw()
RectanglarGlyph draw. degree is 0
Glyph() after draw()
RectanglarGlyph Constructor. degree is 30
```

注意以上的代码，很显然的，在`RectangularGlyph`构造器构造的时候，其调用了父类的构造器，然后父类执行Glyph()中的代码，问题在这里出现。因为其中调用了`draw()`方法，而这个方法在子类中被重载，因此这里其实发生了一个向上转型（思考一下向上转型的定义，在子类中调用父类的方法），并且造成了动态绑定。因此，这里的`draw()`就使用了子类的方法。

这种错误可以避免，只要尽可能的少在构造器中引入方法，采用简单的方式初始化对象就可以了，或者，如果不想让子类继承，那么使用 final 关键字，如果不想让别人看见，使用 private 关键字即可。

# 多态的其他用途

## 协变返回类型

协变返回类型指的是，在导出类中的被覆盖方法可以返回基类方法的返回类型的某种导出类型。

只要按照正常逻辑理解，这是很自然的事情。在SE5中被支持。

协变返回类型指的是这样的实现：

```java
public interface Factory<T> {
    T create();
} //这里的T不一定返回T，还可以是T的导出类型
class Filter extends Part {}

class FueFilter extends Filter {
    public static class Callback implements Factory<Part> {
        public FueFilter create() {
            return new FueFilter(); //导出类型
        }
    }
}
class Belt extends Part {}
class FanBelt extends Belt {
    public static class Callback implements Factory<Part> {
        public FanBelt create() {
            return new FanBelt(); //导出类型
        }
    }
}
```

这样的接口允许实现了这个接口的类返回接口允许的类型和这个类型的导出类型。

# 利用继承的多态进行设计

## 基本范式

总结而言，**使用继承表示行为的差异，使用字段表示状态变化**是使用继承和组合的核心所在。

其**基本范式**概括为：在继承树上通过不断继承来表示更多、更丰富的行为差异，而在客户端（组合模式）中通过调用继承树的基类API，使用字段表示状态变化。如下所示：

```java
/xxx/Starship.java
//those code show how to design with inhert & combine
import static com.mazhangjing.Print.*;
public class Starship {
	private Status status;
	Starship() {
		status = new Status();
	}
	public static void main(String[] args){
		Starship x = new Starship();
		x.changeStatus(new WarnStatus());
		x.action();
		x.changeStatus(new FineStatus());
        //使用字段表示状态变化
		x.action();
	}
	public void changeStatus(Status s){
		this.status = s;
	}
	public void action() {
		status.action();
	}
}
class Status {
	public void action() {
		print("Spaceship not load status unit...");
	}
}
class AlertStatus extends Status {
	public void action() {
		print("AlertStatus action on..");
	}
}
class WarnStatus extends Status {
	public void action() {
		print("WarnStatus action on..");
	}
}
class FineStatus extends Status {}
WarnStatus action on..
Spaceship not load status unit..
```

其中，AlertStatus、WarnStatus、FineStatus都是从Status中导出的类，其代表了行为——Status之间的差异，用来表示不同的状态（继承在实际编程中的最重要作用）。因此，利用继承和组合的技术，将这一状态实例化到某一个具体类中，然后写一些方法对这些有差异状态的基类进行交互，称之为字段表示行为。

这样的话，我们的Spaceship类就仅需要和Status基类的API进行交互,在这个例子中是`action`，而不用管这些不同差异的状态类是如何实现的。这就是多态。

如上所示，当飞船发生紧急情况，我们传递进去了一个向上提升的WarnStatus类，之后使用基类的API和其交互，编译器根本不知道具体的代码和其行为，这为我们提供了巨大的可操纵性和模块接口能力。

继承被用来表示状态差异，而其实例化后，使用一个字段来更改状态，之后通过调用基类API，使用多态来展示其不同状态/子类的内部情况。这样我们有一个清晰的接口（通过多态和向上转型），也有了丰富变化的能力（通过继承和动态绑定、方法重载），而这就是OOP的核心所在。在之后几章，我们将看到在此基础上Java对于接口这种可以让各个继承树在同一API框架下进行协作，从而有效促进代码重用的功能，以及内部类这种从类的实现层面上结合各个类的方式，而正是这些支持，让基于**继承和多态**的面向对象在Java的实现下变得更加灵活、高效。

## 纯继承和扩展继承

对于基类的方法，如果我们在子类中完全复刻一份进行重载，就称之为 `is-a` 关系，在这种关系中，所有的子类都可以替代基类。这是一种理想的情况。

但是，真实情况下，我们总是会对导出类进行各种的扩展，因此，就形成了 `is-like-a` 的关系，在这种关系中，一旦向上转型，那么导出类的扩展方法就难以使用。

## 向下转型和运行时类型识别

通过向下转型，可以获得导出类的类型信息，但是这样不一定是安全的，或者说，通常是很不安全的。使用直接类型转换可以进行向上转型和向下转型：

```java
//show how to use up casting
import static com.mazhangjing.Print.*;
public class Game {
	public static void main(String[] args){
		Cycle[] clist = {new Unicycle(), new Bicycle(), new Tricycle()};
		clist[0].balance(); //Unicycle balance
		clist[1].balance(); //Bicycle balance
		clist[2].balance(); //Cycle balance
		((Cycle)clist[2]).balance(); //up casting
		//((Bicycle)clist[0]).balance(); //down casting
		//Unicycle cannot be cast to Bicycle
	}
}
class Cycle {
	public void ride() {
		print("Cycle ride");
	}
	public void balance() {
		print("Cycle balance");
	}
}
class Unicycle extends Cycle {
	public void ride() {
		print("Unicycle ride");
	}
	public void balance() {
		print("Unicycle balance");
	}
}
class Bicycle extends Cycle {
	public void ride() {
		print("Bicycle ride");
	}
	public void balance() {
		print("Bicycle balance");
	}
}
class Tricycle extends Cycle {
	public void ride() {
		print("Tricycle ride");
	}
}
```

如上的数组，进行了向上转型。而`((Bicycle)clist[0]).balance();`则进行了向下转型(从Cycle到Bicycle)。如果不可以转型，那么回返回一个`ClassCastException`异常。
